# Build an LLM Agent from Scratch

It looks like magic when you witness AI agents autonomously viewing files, making edits, executing commands, and working around errors. However, under the hood, the core loop is surprisingly simple: it runs a continuous cycle that takes user input, generates tool calls and text output, receives feedback from tool execution, feeds the result back to the LLM, and repeats. The complexity lies not in any orchestration, but in the sophisticated reasoning capabilities of the frontier labs' large language models.

To demystify this magic, we'll implement a simple CLI-based coding agent in Python that can read files, write content, edit text, and execute shell commands. Don't worry if you're new to AI agents - this tutorial is designed to be approachable and practical. You'll be amazed at how much you can accomplish with surprisingly little code.

The diagram below illustrates the fundamental agentic loop in action:

 ```mermaid
 graph TD
     Start[User Input] --> Loop[LLM Inference]

     Loop -->|generates| Output[LLM Output]

     Output --> EndsCheck{Ends?}

     EndsCheck -->|Yes| Exit[Break/Exit Loop]

     EndsCheck -->|No| ToolCheck{Tool Calls?}

     ToolCheck -->|Yes| Tools[Execute Tool Calls]
     Tools -->|tool results as new user input| Loop

     ToolCheck -->|No| Loop

     style EndsCheck fill:#faa,stroke:#333,stroke-width:2px
     style ToolCheck fill:#f9f,stroke:#333,stroke-width:2px
     style Tools fill:#bfb,stroke:#333,stroke-width:2px
     style Exit fill:#ddd,stroke:#333,stroke-width:2px
 ```

By the end of this tutorial, we'll have a Python script that's just over a handred lines of code - compact, powerful, and surprisingly capable of giving you the 'Oh look at that!' dopamine hits. I've structured this to be easy to follow along step by step, and I highly encourage you to try it out yourself.

## Prerequisites

You need an Anthropic API key for accessing to Claude

In [3]:
import os
import getpass

api_key = os.getenv("ANTHROPIC_API_KEY")
if not api_key:
    api_key = getpass.getpass("Enter your Anthropic API key: ")
    os.environ["ANTHROPIC_API_KEY"] = api_key# Your agent code here

## The Core Loop

Let's start with the foundation. Every AI agent needs a chat loop that maintains context and handles the back-and-forth between user and AI. Here's our basic structure:

In [20]:
from anthropic import AsyncAnthropic
import json, os, asyncio
from datetime import datetime

client = AsyncAnthropic()

async def loop(system_prompt, messages, user_input):
    messages.append({"role": "user", "content": user_input})

    msg = await client.messages.create(
        max_tokens=2048,
        thinking={"type": "enabled", "budget_tokens": 1024},
        system=[{"type": "text", "text": system_prompt}],
        messages=messages,
        model="claude-sonnet-4-5"
    )

    messages.append({"role": "assistant", "content": msg.content})

    thinking_text = ' '.join(t.thinking for t in msg.content if t.type == 'thinking')
    if thinking_text:
        print(f"💭 {thinking_text}")

    agent_text = ' '.join(t.text for t in msg.content if t.type == 'text')
    if agent_text:
        print(f"🤖 {agent_text}")

async def main():
    messages = []
    system_prompt = f"""Your name is HAL.
You are an interactive CLI tool that helps with software engineering and production operations tasks.
You are running on {str(os.uname())}, today is {datetime.now().strftime('%Y-%m-%d')}
"""

    while True:
        user_input = input("> ")
        if user_input.lower() == "exit":
            break
        await loop(system_prompt, messages, user_input)

await main()

>  exit


This script establishes the core conversation loop of our agent. The loop function handles the core conversation cycle: it adds user input to the message history, sends everything to Claude, processes the response, and displays it. The thinking parameter enables Claude's extended thinking and reasoning capabilities with a 1024-token budget, allowing it to work through complex problems step-by-step thought process. The main function creates an interactive REPL that maintains conversation context across exchanges.

This gives us a working chat interface where we can have fascinating conversations with HAL, after all it embodies vast human knowledge from across the internet. However, HAL remains confined to the realm of text and images—the only way for it to interact with the outer world is through you manually feeding it information such as code, logs, compiler errors, etc., which is not only inefficient but also potentially biased. To bridge this gap and give it the ability to act in the real world, we need to equip it with tools so that it can read files, run programs to get results and errors autonomously.

## Adding Tool Capabilities

As for large language models, the most straightforward way to feed them with external information is to provide them the ability to read text, therefore the first tool we'll implement is file reading. The function below shows file reading in its simplest form:

In [8]:
import aiofiles
from typing import Annotated

async def read_file(filename: Annotated[str, "The path to the file to read"]):
    """Read the whole file"""
    async with aiofiles.open(filename, 'r') as f:
        return {"success": True, "filename": filename, "output": await f.read()}

This function works, but HAL doesn't know its existence, neither does it have access to the Python runtime. We need to bridge the gap between Python functions and jsonschemas that the model can understand.

## The Tool System

Here's where the magic happens. We'll create a decorator that automatically generates the necessary schemas and a toolbox to manage everything:

In [10]:
from typing import Annotated, get_type_hints, get_origin, get_args
from inspect import Parameter, signature
from pydantic import create_model, Field


def tool(f):
    def _process_parameter(name: str, param: Parameter, hints: dict) -> tuple:
        """Process a single parameter into a model field specification."""
        default = ... if param.default == Parameter.empty else param.default
        hint = hints.get(name, param.annotation)

        if get_origin(hint) == Annotated:
            base_type, *metadata = get_args(hint)
            description = next((m for m in metadata if isinstance(m, str)), None)
            return (base_type, Field(default=default, description=description) if description else default)

        return (hint, default)

    hints = get_type_hints(f, include_extras=True)
    model_fields = { name: _process_parameter(name, param, hints) for name, param in signature(f).parameters.items() }

    m = create_model(f'{f.__name__} Input', **model_fields)
    m.run = lambda self: f(**self.model_dump())

    return {
        "name": f.__name__,
        "description": f.__doc__ or f"Tool: {f.__name__}",
        "model": m
    }

class Toolbox:
    def __init__(self, tools):
        self.tools = tools

    def schema(self):
        return [{
            "name": t["name"],
            "description": t["description"],
            "input_schema": t["model"].model_json_schema()
        } for t in self.tools]

    async def run(self, name, input):
        tool = next(t for t in self.tools if t["name"] == name)
        return await tool["model"](**input).run()

At its core, this code creates a bridge between Python functions and AI models. The @tool decorator examines function's signature and automatically generates a JSON schema that describes what inputs the function expects - think of it as creating a user manual for the Python function that the AI can read. The Toolbox class acts like a registry, collecting all these tool schemas and providing a way to execute them when the AI requests it. When HAL wants to use a tool, it sends a structured output that matches the schema, and the toolbox validates the input and runs the corresponding Python function.

**Also don't worry if you can't understand what this function does. Just use this as a utility**

Now we can decorate our file reading function:

In [11]:
@tool
async def read_file(filename: Annotated[str, "The path to the file to read"]):
    """Read the whole file"""
    async with aiofiles.open(filename, 'r') as f:
        return {"success": True, "filename": filename, "output": await f.read()}

Once decorated, we can generate a JSON schema that describes the function to the AI model. Each schema at minimum contains the function's name, description, and parameter specifications. This schema acts as a contract when HAL wants to use a tool, it must structure its output according to this schema, making it the protocol between the HAL and the Python runtime:

In [13]:
toolbox = Toolbox([read_file])
print(json.dumps(toolbox.schema(), indent=2))

[
  {
    "name": "read_file",
    "description": "Read the whole file",
    "input_schema": {
      "properties": {
        "filename": {
          "description": "The path to the file to read",
          "title": "Filename",
          "type": "string"
        }
      },
      "required": [
        "filename"
      ],
      "title": "read_file Input",
      "type": "object"
    }
  }
]


## Connecting Tools to the Loop
Now we need to update our main loop to handle tool calls. When Claude wants to use a tool, it returns special tool_use blocks instead of just text:

In [36]:
async def loop(system_prompt, toolbox, messages, user_input):
    messages.append({"role": "user", "content": user_input})

    while True:
        msg = await client.messages.create(
            max_tokens=2048,
            thinking={"type": "enabled", "budget_tokens": 1024},
            system=[{"type": "text", "text": system_prompt}],
            messages=messages,
            model="claude-sonnet-4-5",
            tools=toolbox.schema()
        )

        messages.append({"role": "assistant", "content": msg.content})

        thinking_text = ' '.join(t.thinking for t in msg.content if t.type == 'thinking')
        if thinking_text:
            print(f"💭 {thinking_text}")

        agent_text = ' '.join(t.text for t in msg.content if t.type == 'text')
        if agent_text:
            print(f"🤖 {agent_text}")

        tools = [t for t in msg.content if t.type == "tool_use"]
        if not tools:
            break

        # Execute all tools and collect results
        results = await asyncio.gather(*[
            toolbox.run(t.name, t.input) for t in tools
        ])

        # Display results and send back to model
        for t, r in zip(tools, results):
            status = "✅" if r.get("success") else "❌"
            print(f"{status} {t.name}:")
            print(json.dumps(r, indent=2))

        messages.append({
            "role": "user",
            "content": [{
                "type": "tool_result",
                "tool_use_id": t.id,
                "content": json.dumps(r)
            } for t, r in zip(tools, results)]
        })

async def run_agent(tools=[]):
    toolbox = Toolbox(tools)
    messages = []
    system_prompt = f"""Your name is HAL.
You are an interactive CLI tool that helps with software engineering and production operations tasks.
You are running on {str(os.uname())}, today is {datetime.now().strftime('%Y-%m-%d')}
"""
    print("enter 'exit' to quit")

    try:
        while True:
            user_input = input("> ")
            if user_input.lower() == "exit":
                print("👋 Goodbye!")
                break
            await loop(system_prompt, toolbox, messages, user_input)
    except (KeyboardInterrupt, EOFError):
        print("👋 Goodbye!")

In [37]:
await run_agent(tools=[read_file])

enter 'exit' to quit


>  exit


👋 Goodbye!


## Adding More Tools
With our infrastructure in place, adding more capabilities becomes straightforward. Let's implement three essential tools: file writing, editing, and shell execution:

In [34]:
@tool
async def write_file(filename: str, content: str):
    """Write the file with the content. This is an overwrite"""
    async with aiofiles.open(filename, 'w') as f:
        await f.write(content)
        return {"success": True, "filename": filename, "output": f"wrote to {filename}"}

@tool
async def edit_file(filename: str, old_text: Annotated[str, "the text to replace"], new_text: Annotated[str, "the text to replace with"]):
    """Edit the file with the new text. Note that the text to replace should only appear once in the file."""
    async with aiofiles.open(filename, 'r') as f:
        content = await f.read()

    if content.count(old_text) != 1:
        return {"success": False, "filename": filename, "output": f"old text appears {content.count(old_text)} times in the file"}

    async with aiofiles.open(filename, 'w') as f:
        await f.write(content.replace(old_text, new_text))

    return {"success": True, "filename": filename, "output": f"edited {filename}"}

@tool
async def shell(command: Annotated[str, "command to execute"], timeout: Annotated[int, "timeout in seconds"] = 30):
    """Execute a bash command"""
    try:
        p = await asyncio.create_subprocess_shell(
            command,
            stdout=asyncio.subprocess.PIPE,
            stderr=asyncio.subprocess.STDOUT
        )
        stdout, _ = await asyncio.wait_for(p.communicate(), timeout)
        return {"success": p.returncode == 0, "command": command, "output": stdout.decode()}
    except asyncio.TimeoutError:
        return {"success": False, "command": command, "output": "Timeout"}

## Putting It All Together
Finally, we wire everything up in our main function

In [39]:
await run_agent(tools=[read_file, write_file, edit_file, shell])

enter 'exit' to quit


>  exit


👋 Goodbye!


What many people don't realise is that frontier models like Claude possess incredible system operations knowledge. Once you provide it with shell access and basic system information, even without extensive prompting, these models demonstrate expert-level operations capabilities. Claude Sonnet models in particular exhibit **persistent, relentless and infectiously optimistic** problem-solving behavior. When asked a question like "how is python3 installed on this machine?", they will systematically explore the filesystem, examine package managers, check environment variables, and execute diagnostic commands until they find a definitive answer.

When commands fail or produce errors, the model treats these as valuable debugging signals, much like an experienced engineer would. It analyses error messages, adjusts its approach, and tries alternative strategies to work around the problem.

While hallucination remains a concern (perhaps one of the most common criticisms of LLMs), the model's ability to execute commands creates a unreasonably effective feedback loop. When its assumptions prove incorrect, the resulting errors quickly ground the model back in reality, forcing it to revise its mental model and try new approaches based on actual system behavior rather than potentially outdated training data or hallucinations.

## MCP MCP MCP

TBD